In [ ]:
!pip install kaggle --upgrade

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sibbir","key":"c9b1f65daec00b23325e5e866738f3b6"}'}

In [ ]:
import os
import zipfile

# Create a Kaggle directory
os.makedirs("/root/.kaggle", exist_ok=True)

# Move kaggle.json to the correct directory
!mv kaggle.json /root/.kaggle/

# Set correct permissions
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d thienkhonghoc/affectnet -p /content

Dataset URL: https://www.kaggle.com/datasets/thienkhonghoc/affectnet
License(s): unknown


In [ ]:
!kaggle datasets download -d thienkhonghoc/affectnet

Dataset URL: https://www.kaggle.com/datasets/thienkhonghoc/affectnet
License(s): unknown
affectnet.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -q /content/affectnet.zip -d /content/affectnet > /dev/null 2>&1

In [ ]:
!pip install torch torchvision timm matplotlib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from collections import Counter
from PIL import Image
import os
import numpy as np
from torch.optim.swa_utils import AveragedModel
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

# Load AffectNet dataset (Ensure paths are correct)
train_data_path = "/content/affectnet/AffectNet/train"
val_data_path = "/content/affectnet/AffectNet/val"
# Optimize CUDA performance
torch.backends.cudnn.benchmark = True

# Improved Data Augmentation
transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.3),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.2), ratio=(0.3, 3.3), value="random"),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# Function to filter out corrupt images
def filter_corrupt_images(dataset):
    valid_samples = []
    for path, label in dataset.samples:
        try:
            _ = Image.open(path).convert("RGB")
            valid_samples.append((path, label))
        except Exception:
            print(f"Corrupt image removed: {path}")
    dataset.samples = valid_samples

# Load datasets
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform)
val_dataset = datasets.ImageFolder(root=val_data_path, transform=transform)

# Filter corrupt images
filter_corrupt_images(train_dataset)
filter_corrupt_images(val_dataset)

# Compute class weights efficiently
class_counts = np.bincount(train_dataset.targets)
weights = torch.tensor(len(train_dataset) / (len(class_counts) * class_counts), dtype=torch.float).to(device)

print(f"Class Weights: {weights}")

# Create data loaders
num_workers = min(4, os.cpu_count() // 2)  # Dynamically assign workers
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=num_workers, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=num_workers, pin_memory=True, persistent_workers=True)

# Load ConvNeXt-Small model
model = models.convnext_small(weights=models.ConvNeXt_Small_Weights.IMAGENET1K_V1)
model.classifier[2] = nn.Linear(model.classifier[2].in_features, 8)
model = model.to(device)

# Stochastic Weight Averaging (SWA)
swa_model = AveragedModel(model)

print("Model initialized!")

# Define loss function & optimizer
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

# Early Stopping & Mixed Precision
scaler = torch.cuda.amp.GradScaler()
best_val_acc = 0.0
early_stopping_patience = 5
epochs_without_improvement = 0

# Label smoothing
def smooth_labels(labels, smoothing=0.1, num_classes=8):
    confidence = 1.0 - smoothing
    label_matrix = torch.full((labels.size(0), num_classes), smoothing / (num_classes - 1)).to(device)
    label_matrix.scatter_(1, labels.unsqueeze(1), confidence)
    return label_matrix

# Training loop
num_epochs = 20
save_interval = 5
accumulation_steps = 2  # Gradient accumulation

print("\nStarting Training...\n")
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0
    correct_train, total_train = 0, 0

    # Training phase
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images)

            # Apply MixUp Augmentation
            alpha = 0.2
            lam = np.random.beta(alpha, alpha)
            index = torch.randperm(images.size(0)).to(device)
            mixed_images = lam * images + (1 - lam) * images[index]
            mixed_labels = lam * smooth_labels(labels) + (1 - lam) * smooth_labels(labels[index])

            loss = F.kl_div(F.log_softmax(outputs, dim=1), mixed_labels, reduction="batchmean")

        scaler.scale(loss).backward()

        # Gradient accumulation
        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_accuracy = 100 * correct_train / total_train

    # Validation phase
    model.eval()
    correct_val, total_val = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_accuracy = 100 * correct_val / total_val
    scheduler.step()

    # Update SWA model
    swa_model.update_parameters(model)

    # Print epoch summary
    print(f"Epoch [{epoch}/{num_epochs}]")
    print(f"   Loss: {running_loss:.4f}")
    print(f"   Train Accuracy: {train_accuracy:.2f}%")
    print(f"   Validation Accuracy: {val_accuracy:.2f}%\n")

    # Save model every 5 epochs
    if epoch % save_interval == 0:
        save_path = f"affectnet_convnext_epoch{epoch}.pt"
        torch.save(model.state_dict(), save_path)
        print(f"Model saved: {save_path}\n")

    # Early stopping with checkpointing
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        epochs_without_improvement = 0
        torch.save(model.state_dict(), "affectnet_best.pt")
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= early_stopping_patience:
        print(f"Early stopping triggered. Best validation accuracy: {best_val_acc:.2f}%")
        break

# Save final SWA model
torch.optim.swa_utils.update_bn(train_loader, swa_model)
torch.save(swa_model.state_dict(), "affectnet_final_swa.pt")
print("\nTraining complete! SWA model saved.")

Using device: cuda
Loading and preprocessing datasets...
Class distribution: [5000 3803 5000 5000 5000 5000 5000 3750]
Class weights: [0.9250603  1.21622443 0.9250603  0.9250603  0.9250603  0.9250603
 0.9250603  1.23341374]
Initializing model...


<ipython-input-18-207ef0692104>:85: UserWarning: Argument(s) 'height, width' are not valid for transform RandomResizedCrop
  A.RandomResizedCrop(height=224, width=224, scale=(0.8, 1.0), size=(224, 224)), # Added size parameter
<ipython-input-18-207ef0692104>:97: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width' are not valid for transform CoarseDropout
  A.CoarseDropout(


Starting training...


<ipython-input-18-207ef0692104>:254: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-18-207ef0692104>:283: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch [1/30], Batch [50/1173], Loss: 1.5806, LR: 0.000020
Epoch [1/30], Batch [100/1173], Loss: 2.2326, LR: 0.000020
Epoch [1/30], Batch [150/1173], Loss: 2.6892, LR: 0.000020
Epoch [1/30], Batch [200/1173], Loss: 2.0897, LR: 0.000020
Epoch [1/30], Batch [250/1173], Loss: 1.8558, LR: 0.000020
Epoch [1/30], Batch [300/1173], Loss: 2.0571, LR: 0.000020
Epoch [1/30], Batch [350/1173], Loss: 2.0974, LR: 0.000020
Epoch [1/30], Batch [400/1173], Loss: 1.8550, LR: 0.000020
Epoch [1/30], Batch [450/1173], Loss: 2.1184, LR: 0.000021
Epoch [1/30], Batch [500/1173], Loss: 2.0097, LR: 0.000021
Epoch [1/30], Batch [550/1173], Loss: 1.9710, LR: 0.000021
Epoch [1/30], Batch [600/1173], Loss: 1.9536, LR: 0.000021
Epoch [1/30], Batch [650/1173], Loss: 1.9366, LR: 0.000021
Epoch [1/30], Batch [700/1173], Loss: 1.7858, LR: 0.000021
Epoch [1/30], Batch [750/1173], Loss: 1.6621, LR: 0.000021
Epoch [1/30], Batch [800/1173], Loss: 1.8202, LR: 0.000022
Epoch [1/30], Batch [850/1173], Loss: 1.9346, LR: 0.00002